## 探查被职业特性塑造的睡眠模式

职业是塑造个人生活模式的核心因素。在当代职场环境中，不同职业的工作节奏、压力源与作息规律正显著影响个体的睡眠模式——程序员可能因项目上线熬夜调试代码，医护人员需轮班应对突发状况，销售从业者常因业绩压力失眠。这些职业特性并非孤立存在，而是通过"睡眠时长""压力感知""身体活动"等具体行为，最终反映为差异化的睡眠健康状态。

基于上述背景，我们聚焦于非学生群体的在职人员（数据集涵盖IT、医疗、教育、金融等8个行业共400条合成数据），尝试通过K-means聚类算法，挖掘不同职业人群在睡眠时长、焦虑水平、运动习惯等核心变量上的聚集规律，进而识别具有相似特征的睡眠健康亚群，为针对性干预提供依据。

---

### 1. 数据预处理

在对数据进行预处理的过程中，除了对分类变量进行独热编码以外，有选择地drop了PersonID与Blood Pressure这两列（血压列的处理相对麻烦，并且做过保留的尝试，发现对分类的作用并不大，甚至使聚类更加困难），并且对BMI处理中发现数据中的Normal与Normal Weight应该是同一类别，于是还进行了合并化处理。

[../src/03_occupation_and_sleep/data_preprocessing.py](../src/03_occupation_and_sleep/data_preprocessing.py)

![../reports/03_occupation_and_sleep/sleep_disorder_correlation_matrix.png](../reports/03_occupation_and_sleep/sleep_disorder_correlation_matrix.png)

在正式开展K-means聚类分析前，绘制了特征关系矩阵热力图以进行初步探索。该热力图基于变量间的相关系数，用于直观呈现各特征（如睡眠时长、压力水平、运动频率等）之间的线性关联模式。

通过观察热力图的颜色分布，我们能够快速识别出特征间的关联性强度——例如，睡眠时长与睡眠质量呈现明显的正相关趋势，而压力水平与部分职业之间也存在潜在关联。这一分析帮助我们发现部分变量间存在信息重叠（如高度相关的特征可能传递相似信号），并验证了所选变量与睡眠健康问题的逻辑一致性（如高压人群更易伴随睡眠质量下降）。

该步骤为后续聚类分析提供了两项关键支持：其一，通过识别冗余特征优化了输入变量选择，避免无关干扰；其二，明确了与睡眠健康强相关的核心特征组合，使聚类结果更可能反映实际人群的分组规律。整体上，热力图分析增强了后续聚类算法对睡眠健康差异模式的识别有效性。

---

### 2. K-means聚类

我们采用肘部法则和轮廓系数来确定最佳K值。

[../src/03_occupation_and_sleep/k-means.py](../src/03_occupation_and_sleep/k-means.py)

![../reports/03_occupation_and_sleep/elbow_method.png](../reports/03_occupation_and_sleep/elbow_method.png)

手肘图的拐点其实算不上明显，下结合轮廓系数分析。

![../reports/03_occupation_and_sleep/silhouette_score.png](../reports/03_occupation_and_sleep/silhouette_score.png)

![../reports/03_occupation_and_sleep/silhouette_score_vs_k.png](../reports/03_occupation_and_sleep/silhouette_score_vs_k.png)

在初步尝试不同聚类数（K值）的过程中，我观察到当K值设置较高时，虽然算法理论上能识别出更细致的群体划分，但实际上产生了过多碎片化的微小簇群。这些簇群往往仅包含极少量样本且特征差异微小，既缺乏实际解释意义，也增加了结果应用的操作难度——难以从这些过于细碎的分类中提炼出具有明确指导价值的共性特征。

经过这一观察，我们反思了原始特征集合的维度问题：原始数据包做完预处理后我选择了24个变量，其中部分信息（如心率）虽与健康相关，但与本次研究聚焦的"睡眠质量及影响因素"核心问题存在一定距离，且可能与其它变量存在信息重叠。因此采取了特征精简策略，重点保留与研究目标直接相关的核心维度——睡眠基础指标（睡眠时长、睡眠质量）、主观压力感知、运动强度、职业类型（作为关键生活情境变量）以及BMI体重类别（作为基础健康参照）。

这一特征筛选过程基于两点：1.所选特征直接关联"睡眠健康"的研究主题（如睡眠时长与质量是核心因变量，压力与运动是已知的影响因素）；2.职业与BMI作为重要的背景变量，既能反映不同生活情境对睡眠的影响（如职业特性决定的作息规律），又能控制基础健康状态的干扰（如体重类别可能关联的代谢差异）。通过人为降低特征维度，在保留关键信息的同时减少了变量间的冗余，使后续聚类分析更可能捕捉到具有实际意义的聚类，或者说人群分组。

这一调整旨在平衡模型的复杂性与解释力，验证精简后的特征集合是否能更清晰地揭示不同职场人群的睡眠健康模式差异。

[../src/03_occupation_and_sleep/reduced_k-means.py](../src/03_occupation_and_sleep/reduced_k-means.py)

![../reports/03_occupation_and_sleep/new_silhouette_score.png](../reports/03_occupation_and_sleep/new_silhouette_score.png)

![../reports/03_occupation_and_sleep/new_silhouette_score_vs_k.png](../reports/03_occupation_and_sleep/new_silhouette_score_vs_k.png)

在完成特征精简后，我们进行了不同K值（聚类数）下的轮廓系数评估，以客观衡量聚类结果的合理性与分离度。分析过程中观察到，随着K值增大（即尝试划分更多聚类簇），轮廓系数整体呈现上升趋势——这一现象符合理论预期，因为更高的K值允许算法识别更细微的群体差异，从而在数学上提升各簇内紧密度与簇间分离度的比值。
但值得注意的是，与精简特征前的初步分析相比，当前特征集合下的轮廓系数曲线在拐点之后显示出更显著的提升幅度。具体而言，当K值超过某一临界点后，精简特征版本的轮廓系数不仅绝对值更高，而且各簇的结构清晰度明显改善，这表明减少冗余变量后，算法能更有效地捕捉数据中真实存在的群体分化模式。基于这一发现，我们认为在当前精简特征的基础上进行正式K-means聚类分析的确更为合理。

选择 k = 6 进行聚类分析，并绘制睡眠时长和睡眠质量的箱线图。

![../reports/03_occupation_and_sleep/sleep_duration.png](../reports/03_occupation_and_sleep/sleep_duration.png)

![../reports/03_occupation_and_sleep/sleep_quality.png](../reports/03_occupation_and_sleep/sleep_quality.png)

通过对比睡眠时长与睡眠质量两个维度的箱线图分布，我们可以清晰观察到不同聚类群体在睡眠健康状态上的显著差异。这一可视化分析帮助我们验证了聚类算法是否成功捕捉到了具有实际意义的睡眠模式分组。

具体而言，簇0和簇5在两个指标上均表现突出——不仅睡眠时长明显超过7小时的健康基准线（表现为箱体整体位置偏上且中位数超越红色虚线），睡眠质量评分也普遍高于7.5分的良好阈值（箱体的中位线和上边缘均位于红色参考线以上）。这种"双高"特征表明这两个簇很可能代表睡眠质量最佳的群体，其睡眠模式在量与质两个维度都达到了健康标准。

与之形成鲜明对比的是簇1和簇3，这两个群体的睡眠状况在两个指标上都处于劣势：睡眠时长普遍低于7小时基准（箱体位置整体偏下，中位数线明显在红色虚线下方），同时睡眠质量评分也较低（多数数据点低于7.5分，且整体分布区间偏向低端）。这种"双低"模式提示我们，这两个簇可能代表了睡眠健康问题较为突出的群体，其睡眠不足与低质量体验可能相互关联。

簇2和簇4则呈现出相对平衡的中间状态——簇2睡眠时长基本维持在7小时左右（箱体中位数接近健康阈值线，既有超过也有未达标的分布）簇4的睡眠时长更好一些为“长睡眠”；二者睡眠质量评分也处于中等偏上水平（多数数据落在7——8分左右，未达到全员优秀但也不属于低质范畴）。这种特征表明这两个簇可能代表睡眠状况基本正常的普通群体，其睡眠模式虽未达到最优，但也没有明显的健康风险。

通过这种直观的分布对比，我们可以合理推断聚类算法确实识别出了具有实际差异的睡眠健康亚群，且这些分组与我们基于常识的健康标准（如7小时睡眠为基准线）具有良好对应关系。

在探索压力与睡眠质量的关系时，基于我的日常经验认知提出假设：随着压力水平的提升，个体的主观睡眠质量往往呈现下降趋势——这种负相关关系符合多数人的直观感受：当生活或工作压力增大时，夜间入睡困难、睡眠深度降低、早八疲惫感增强。

为了验证这一假设并直观呈现数据中的潜在模式，我们选择了散点图（气泡图）作为主要可视化形式，将压力水平设为横轴、睡眠质量设为纵轴，并通过颜色区分不同的聚类群体（hue='Cluster'）。这种图表设计不仅能展示两个连续变量之间的关系强度与方向，还能通过点的分布密度识别可能的群体聚集特征。

![../reports/03_occupation_and_sleep/stress_level_vs_sleep_quality.png](../reports/03_occupation_and_sleep/stress_level_vs_sleep_quality.png)

从图中趋势和聚类分布可知，压力水平与睡眠质量呈负相关——压力越高，睡眠质量越差。同时，不同聚类（颜色）的群体在“压力-睡眠”关系上有细微差异（如Cluster 5在低压力下睡眠极佳，Cluster 1在高压力下睡眠极差），但整体负相关的趋势一致

除了上述分析外，我们比较在意职业的情况——到底什么职业被压榨得如此辛苦（如压力水平很高）连觉也睡不好，为此我们计算每个职业在各个簇中的数量和比例，并用热力图能清晰地看到每个职业主要分布在哪些簇中，并判断哪些簇可以代表该职业群体。

![../reports/03_occupation_and_sleep/occupation_distribution.png](../reports/03_occupation_and_sleep/occupation_distribution.png)

由于有些职业样本数太少了，所以在统计意义上不足以相信其分簇得到的特征，因此下面的分析我们都排除掉Manager（样本数1）、Scientist（样本数4）、Software Engineer（样本数4）。
总结剩余的职业，我们可以看出：
* 簇0：Engineer；
* 簇1：销售类职业（Salesperson & SalesRepresentative）；
* 簇2：Teacher、Accountant、Lawyer；
* 簇3和簇5都对应的是两类护士Nurse；
* 簇1和簇4都对应的是两类医生Doctor；

出于自己对身边的医学生在不同科室实习出现日版夜班、连续值夜等了解，我再次对每个职业睡眠时长进行箱线图分析，可推测医生与护士因轮班和不规律的工作时间而导致很难在k=6时找到一个代表簇——即医护工作者的睡眠变化很大。

![../reports/03_occupation_and_sleep/sleep_duration_by_occupation.png](../reports/03_occupation_and_sleep/sleep_duration_by_occupation.png)

对“不同职业睡眠时长”的箱线图分析
该箱线图展示了11类职业（Sales Representative、Scientist、Salesperson、Nurse、Teacher、Software Engineer、Manager、Accountant、Lawyer、Doctor、Engineer）的睡眠时长分布，结合健康睡眠阈值（7小时，红色虚线）和箱线图的统计特征，可从以下维度客观分析行业间的睡眠时长差异（中位数与分布）：
* 护士（Nurse）：箱体最长，中位数约6.5小时，但整体分布跨度大（从约6.0到8.2小时）。结合背景（轮班、不规律工作），说明护士群体的睡眠时长个体差异极大——部分人因工作强度（如连续夜班）睡眠严重不足，部分人可能通过休息补偿达到较高时长。
* 医生（Doctor）：箱体长度次之，中位数约7.6小时，分布从6.0到8.2小时。异常值（如8.0小时以上的点）提示部分医生睡眠充足，但仍有低至6.0小时的极端值，反映医疗工作的不规律性对睡眠的影响。
* 工程师（Engineer）：中位数约8.3小时，箱体较短（离散程度小），多数数据集中在7.7~8.5小时，睡眠时长整体充足且稳定，可能与工作节奏相对规律有关。
* 销售类（Sales Representative、Salesperson）：箱体短且中位数低（约5.9、6.4小时），睡眠时长普遍不足且个体差异小，可能与工作时间灵活性低、压力大导致睡眠被压缩有关。
* 办公室职业（Scientist、Teacher、Software Engineer、Manager、Accountant、Lawyer）：睡眠时长介于上述极端之间，其中：
* Scientist（科学家）、Accountant（会计）中位数低（约6.0、6.2小时），睡眠不足风险高；
* Teacher（教师）、Software Engineer（软件工程师）、Lawyer（律师）分布跨越7小时，群体内睡眠差异大（如教师有高至7.2小时的点，也有低至6.3小时的异常值）。

同时我们也考虑考虑异常值与工作特征的关联：
箱线图中的圆圈（异常值）代表偏离群体分布的极端值：
* 护士、医生的异常值（如护士的8.2小时、医生的8.0小时）可能对应“休息日”或“低工作负荷期”的睡眠补偿；（结合“医护工作者轮班、不规律工作”的背景，图中护士、医生的大箱体（高离散度）和跨越健康阈值的分布，验证了“医疗工作对睡眠时长的稳定性影响显著”——轮班制导致个体间睡眠策略（补偿/剥夺）差异大，群体内睡眠时长波动剧烈。）
* 会计的异常值（6.3小时）可能对应“高强度工作期”（如月末、年报季）的睡眠剥夺；
* 销售代表的低异常值（5.9小时）可能反映“持续高压工作”对睡眠的长期挤压。

---

### 3. 总结

**通过聚类分析，我们将研究对象（不同职业人群）划分为6个具有显著特征的聚类群体，各群体的核心特征可从样本分布、BMI健康状态、生活方式（睡眠、活动、压力）等维度总结：**

* 群体0——（Engineer）：
特征为“正常睡眠（8.4小时）+ 高质量睡眠 + 低压 + 低活动”。该群体职业以工程师为主，生活方式健康（睡眠充足、压力低），且BMI水平良好。

* 群体1（Salesperson销售类、部分Doctor(46.5%)）：
特征为“短睡眠（6.3小时）+ 低质量睡眠 + 高压 + 低活动 + 高BMI异常”。销售人员群体普遍面临睡眠不足、压力大、活动量低的问题，导致BMI异常率较高。

* 群体2（Accountant & Lawyer & Teacher）：
特征为“正常睡眠（7.1小时）+ 高质量睡眠 + 低压 + 低活动”。该群体（会计、律师、教师）睡眠和生活方式相对健康，但仍有近1/3存在BMI异常，可能与职业特性（如久坐、工作压力类型）有关。

* 群体3（Nurse(43.8%)）：
特征为“短睡眠（6.1小时）+ 低质量睡眠 + 高压 + 高活动 + 高BMI异常”。该群体（护士）睡眠严重不足、压力大、活动量高，但BMI仍异常（可能为肥胖或过瘦），职业压力和生活节奏对健康影响显著。

* 群体4（Doctor(45.1%)）：
特征为“正常睡眠（7.7小时）+ 高质量睡眠 + 低压 + 高活动 + 正常BMI”。医生群体虽然活动量高，但睡眠充足、压力低，BMI保持正常，职业特性（如职业素养、健康管理意识）可能起积极作用。

* 群体5（Nurse(46.6%)）：
特征为“正常睡眠（8.1小时）+ 高质量睡眠 + 低压 + 高活动 + 高BMI异常”。该群体（护士）睡眠充足、压力低，但活动量高且BMI异常，可能与职业环境（如饮食、作息规律）有关。

#### 职业与健康的关联洞察

从职业维度看，不同职业的BMI健康状态和生活方式存在显著差异：
工程师、医生：BMI异常率低（0%），核心优势是“睡眠充足+高质量+低压”，说明职业的工作节奏、压力管理和健康意识可能有助于维持BMI健康。
销售人员：BMI异常率高（59.6%），核心问题是“短睡眠+低质量+高压+低活动”，职业的销售压力、工作时间不规律可能导致生活方式失衡。
会计、律师、教师：BMI异常率中等（31.8%），虽睡眠健康，但“低活动”可能是隐患，需关注久坐对健康的影响。
护士：BMI异常率高（群体3为100%，群体5为83.7%），尽管部分群体睡眠充足，但“高活动”或“职业压力”仍导致BMI异常，提示护士职业的健康管理需针对性优化（如饮食、作息调整）。

#### 生活方式对BMI的影响
聚类结果验证了“睡眠+压力+活动量”是影响BMI健康的关键因素：
睡眠充足、高质量睡眠、低压的人群（如工程师、一部分医生），BMI异常率低；
睡眠不足）、低质量睡眠、高压的人群（如销售人员、部分护士），BMI异常率高；
活动量的影响具有复杂性：低活动可能增加BMI异常风险（如群体1），但高活动也可能伴随BMI异常（如群体3、5、可能是压力肥等等），需结合睡眠、压力等因素综合分析。


#### 小建议

尽管职业的工作节律（如医护轮班、销售出差、白领久坐）难以彻底改变，但通过“微习惯调整”（如碎片化运动——肩颈放松拉伸、睡眠优化）、“饮食管理”（如健康餐包、营养替换——避免压力性进食、无意识暴食）、“压力释放”（如正念冥想、假期运动），可在工作节奏中嵌入健康行为，逐步改善BMI和整体健康状态。同时，健康型职业的经验（如工程师的规律睡眠、医生的高活动量管理）可作为模板，向其他职业推广，实现“工作与健康的平衡”。